# Text generation with an RNN

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


## Lab Task 1

In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Lab Task 2

In [5]:
import nltk

In [6]:
# 'punkt' package is a pre-trained model used by NLTK for tokenizing sentences

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from nltk.tokenize import word_tokenize
# Extract and sort unique words from the text using set function

# Use word_tokenize to get word level tokens
text = text.lower()
vocab = sorted(set(word_tokenize(text)))
print(f'{len(vocab)} unique words')

12443 unique words


In [8]:
# Print the Vocabulary
print(vocab)

['!', '$', '&', "'", "'aged", "'alas", "'all", "'alla", "'almost", "'an", "'ay", "'banished", "'banishment", "'be", "'beseech", "'better", "'bless", "'bout", "'bove", "'broad", "'brutus", "'by", "'celsa", "'cerns", "'charge", "'citizens", "'clarence", "'come", "'commend", "'con", "'content", "'coriolanus", "'courage", "'cucullus", "'cum", "'d", "'dear", "'death", "'deny", "'do", "'dost", "'dove", "'em", "'ever", "'fair", "'faith", "'fall'st", "'farewell", "'fine", "'flayed", "'for", "'fore", "'forgive", "'frets", "'gainst", "'girdling", "'go", "'god", "'good", "'grace", "'greed", "'have", "'havior", "'he", "'head", "'heart", "'hell", "'her", "'hic", "'hold", "'honour", "'if", "'ifs", "'is", "'it", "'jack", "'jesu", "'judgment", "'jump", "'king", "'knocking", "'lady", "'larum", "'larums", "'lay", "'leges", "'leven", "'ll", "'lo", "'long", "'longeth", "'longs", "'look", "'lord", "'lordship", "'love", "'m", "'madam", "'man", "'margaret", "'marry", "'mildly", "'mong", "'mongst", "'music", 

## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

Now create the `tf.keras.layers.StringLookup` layer:

In [9]:
ids_from_words = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

You can `tf.strings.reduce_join` to join the characters back into strings.

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [10]:
all_ids = ids_from_words(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([   0, 5516, 8572, ..., 4588,  198,    0])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
words_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [13]:
for ids in ids_dataset.take(10):
    print(words_from_ids(ids).numpy().decode('utf-8'))

[UNK]
i
r
[UNK]
t
[UNK]
c
i
t
i


In [14]:
seq_length = 1000

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(words_from_ids(seq))

tf.Tensor([b'[UNK]' b'i' b'r' ... b'[UNK]' b'[UNK]' b'[UNK]'], shape=(1001,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1)

In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"[UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK][UNK][UNK]or[UNK][UNK][UNK][UNK][UNK][UNK]roc[UNK][UNK][UNK][UNK]a[UNK][UNK][UNK][UNK][UNK]rt[UNK][UNK]r,[UNK][UNK][UNK]ar[UNK][UNK][UNK][UNK][UNK][UNK][UNK]a[UNK].[UNK][UNK]all:[UNK][UNK][UNK][UNK]a[UNK],[UNK][UNK][UNK][UNK]a[UNK].[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK]o[UNK][UNK]ar[UNK][UNK]all[UNK]r[UNK][UNK]ol[UNK][UNK][UNK][UNK]rat[UNK][UNK]r[UNK]to[UNK][UNK]i[UNK][UNK]t[UNK]a[UNK][UNK]to[UNK][UNK]a[UNK]i[UNK][UNK]?[UNK][UNK]all:[UNK]r[UNK][UNK]ol[UNK][UNK][UNK].[UNK]r[UNK][UNK]ol[UNK][UNK][UNK].[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK]ir[UNK]t,[UNK][UNK]o[UNK][UNK][UNK][UNK]o[UNK][UNK]cai[UNK][UNK][UNK][UNK]arci[UNK][UNK][UNK]i[UNK][UNK]c[UNK]i[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK]to[UNK]t[UNK][UNK][UNK][UNK][UNK]o[UNK]l[UNK].[UNK][UNK]all:[UNK][UNK][UNK][UNK][UNK][UNK]o[UNK]'t,[UNK][UNK][UNK][UNK][UNK][UNK]o[UNK]'t.[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK]l[UNK]t[UNK][UNK][UN

For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(1000,), dtype=tf.int64, name=None), TensorSpec(shape=(1000,), dtype=tf.int64, name=None))>

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"[UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK][UNK][UNK]or[UNK][UNK][UNK][UNK][UNK][UNK]roc[UNK][UNK][UNK][UNK]a[UNK][UNK][UNK][UNK][UNK]rt[UNK][UNK]r,[UNK][UNK][UNK]ar[UNK][UNK][UNK][UNK][UNK][UNK][UNK]a[UNK].[UNK][UNK]all:[UNK][UNK][UNK][UNK]a[UNK],[UNK][UNK][UNK][UNK]a[UNK].[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK]o[UNK][UNK]ar[UNK][UNK]all[UNK]r[UNK][UNK]ol[UNK][UNK][UNK][UNK]rat[UNK][UNK]r[UNK]to[UNK][UNK]i[UNK][UNK]t[UNK]a[UNK][UNK]to[UNK][UNK]a[UNK]i[UNK][UNK]?[UNK][UNK]all:[UNK]r[UNK][UNK]ol[UNK][UNK][UNK].[UNK]r[UNK][UNK]ol[UNK][UNK][UNK].[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK][UNK]ir[UNK]t,[UNK][UNK]o[UNK][UNK][UNK][UNK]o[UNK][UNK]cai[UNK][UNK][UNK][UNK]arci[UNK][UNK][UNK]i[UNK][UNK]c[UNK]i[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK]to[UNK]t[UNK][UNK][UNK][UNK][UNK]o[UNK]l[UNK].[UNK][UNK]all:[UNK][UNK][UNK][UNK][UNK][UNK]o[UNK]'t,[UNK][UNK][UNK][UNK][UNK][UNK]o[UNK]'t.[UNK][UNK][UNK]ir[UNK]t[UNK]citi[UNK][UNK][UNK]:[UNK]l[UNK]t[UNK]

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 1000), dtype=tf.int64, name=None), TensorSpec(shape=(64, 1000), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_words.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 1000, 12444) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3185664   
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  12755100  
                                                                 
Total params: 19879068 (75.83 MB)
Trainable params: 19879068 (75.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [30]:
sampled_indices

array([ 1510, 11912,  2493,  8406,  3910,   102,  1785, 10793,  1802,
        2633,  7220, 11829,  9310,  3043,  3565, 10693, 11346,  4025,
       10850,   184,  5048,  3716,  1548,  8917,  5621, 12339,  8689,
        4192,  6483,  8414, 11200,  4409, 11289, 11292,  4353,  9259,
         490,   990,  4690,  8406,  6778,  7472, 10077,   590, 11950,
       10991,  6202,  8070,    16, 11575,  5656,  7535,  1659, 10710,
       10462,  2551,   626,   708,  4543,  1829,  4031,   995,  5151,
        9803,  1643,  9870,  4686,   891,  2334,  7129,  8157,  8621,
        8493, 10125,  8179, 10053,  2271,  6162,  5681, 11393,  3998,
        7578, 12232,  5122,  7253,  3327,  7169,  5658,   557,  3678,
        7210, 12044, 10850, 10990,   391,  8125, 12406,   407,  6404,
        7127,  9143,  1060,  6259,  8317,  2557, 12308,  7571,    62,
        3735,  9398,  8260,  3977,  6667,  5909,   457,  4018,  7630,
        5840,   139,  5184,  9599,  8594,  1518,  6308,  1225, 11690,
        4793,  2509,

In [31]:
len(sampled_indices)

1000

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"li[UNK][UNK],[UNK]o[UNK][UNK]o[UNK][UNK]t[UNK][UNK][UNK][UNK]oor[UNK][UNK]o[UNK][UNK][UNK]r[UNK]at[UNK],[UNK][UNK][UNK]al[UNK][UNK]it[UNK][UNK]a[UNK]rig[UNK]t[UNK]o[UNK][UNK][UNK][UNK]i[UNK][UNK][UNK]a[UNK][UNK]at[UNK]l[UNK][UNK][UNK][UNK][UNK]argai[UNK][UNK]to[UNK][UNK][UNK]gro[UNK][UNK]i[UNK]g[UNK][UNK][UNK]at[UNK]![UNK]co[UNK][UNK],[UNK][UNK]itt[UNK]r[UNK]co[UNK][UNK][UNK]ct,[UNK]co[UNK][UNK],[UNK][UNK][UNK][UNK]a[UNK]o[UNK]r[UNK][UNK]g[UNK]i[UNK][UNK]![UNK]t[UNK]o[UNK][UNK][UNK][UNK][UNK][UNK][UNK]rat[UNK][UNK][UNK]ilot,[UNK][UNK]o[UNK][UNK]at[UNK]o[UNK]c[UNK][UNK]r[UNK][UNK][UNK]o[UNK][UNK]t[UNK][UNK][UNK][UNK]a[UNK][UNK]i[UNK]g[UNK]roc[UNK][UNK][UNK]t[UNK][UNK][UNK][UNK][UNK]a[UNK][UNK]ic[UNK][UNK][UNK][UNK]ar[UNK][UNK][UNK]ar[UNK]![UNK][UNK][UNK]r[UNK]'[UNK][UNK]to[UNK][UNK][UNK][UNK]lo[UNK][UNK]![UNK]o[UNK]tr[UNK][UNK][UNK]a[UNK]ot[UNK][UNK]car[UNK]![UNK]t[UNK][UNK][UNK][UNK]r[UNK]g[UNK][UNK]ar[UNK][UNK][UNK][UNK]ic[UNK].[UNK]t[UNK][UNK][UNK][UNK][UNK]it[UNK][UNK]a[UN

In [33]:
len(text_from_ids(sampled_indices).numpy())

6962

In [34]:
len("E'R;.tMfBjqu'?:rifzeqz\no-? BFXWHi.B3bgay\n;IE,Qi'yAj&XIQYpuMpwOo.Czz\n.BD[UNK]OWRlsoqkXE:Ksl-;LnzWMWbyifj!")

104

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 1000, 12444)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(9.430554, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

12463.435

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [40]:
EPOCHS = 20

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
17/17 [==============================] - 39s 2s/step - loss: 3.8488
Epoch 2/20
17/17 [==============================] - 36s 2s/step - loss: 1.6725
Epoch 3/20
17/17 [==============================] - 38s 2s/step - loss: 1.6107
Epoch 4/20
17/17 [==============================] - 38s 2s/step - loss: 1.6013
Epoch 5/20
17/17 [==============================] - 38s 2s/step - loss: 1.5957
Epoch 6/20
17/17 [==============================] - 39s 2s/step - loss: 1.5870
Epoch 7/20
17/17 [==============================] - 38s 2s/step - loss: 1.5712
Epoch 8/20
17/17 [==============================] - 38s 2s/step - loss: 1.5577
Epoch 9/20
17/17 [==============================] - 40s 2s/step - loss: 1.5439
Epoch 10/20
17/17 [==============================] - 38s 2s/step - loss: 1.5289
Epoch 11/20
17/17 [==============================] - 38s 2s/step - loss: 1.5169
Epoch 12/20
17/17 [==============================] - 39s 2s/step - loss: 1.5087
Epoch 13/20
17/17 [==============================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, words_from_ids, ids_from_words, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.words_from_ids = words_from_ids
    self.ids_from_words = ids_from_words

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_words(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_words.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_words(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.words_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, words_from_ids, ids_from_words)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:.otiracitatgar;;ollat.t'lllotargat,:,to,laco:it:ig,:,ooirgtoig'lglirloralorort::oirala:t,:iiirolliicitattaoillllrorillar,,illilg:oo:ritot,ig,ricgoor;:oloargrittot,otairtitarit,g,icit;oiticitorl.:ortali'titcit,or.oiigit.oor'ititrriglatrlar'lllalititigatiti':?'llllic.;oiolallot;:rt,'talt?t'o:?laa:;'ali'g,;otorlitataio::ag'lllor,itillarg,ocait,igor,or,t'tr,'lotlatt'oratto?::oricar:rtag.:otto:orlar:'t,'t,'ag.o:rorooraorgot.rcoort,,,oot.oolalit'tlaoragatii,:it;..'g,rart,;,it.ll'torat,'tar!it:oailllto:atlo:,.:o:,'taori:.oratat,otatoiotolitg:?'tarilao.:::ootcar,illolat:ogiio:.'rcatotiorc:'?ot:oor,oilllat?:itiot,'g'itic,ootoilaglotall,'g,,'a,.oilitc,'g.or,'t,rgia.'t:ilo:tarlorari,o:'rtoort!o:?'to'igo:,,ot'orito:orrar'or'cat.?cttooitcoolltocilllioo:iago::r,i'a:olartriil:oriclt:tittiotitilat,.'llit.or.'lall.:?itilcogrotioilal.oot?oratoorg.'cor,,'tgillorigil.:ooriicitilllo:'clt,itati:'catilario..oig,'t:iarloct'or.raa,oortoll,:igiolo'g,ricalal.!licg,?rt,ra::i,::,'tilatratt.ratt:'t,olilliil'l

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:otcarotcotatroorti::illt'irillitri:ior.iiiocor,itill'ittotraali:o't,a:ota'tcticliclor,'litg.oot'i'llllotrr:'igogl,i::'taatlo,tootollitotat,.'lllaci;:ort,:io:tiagligtititoo;rtoratita,.oallot,,:lit.ilor.:rarcio,,'llllarar.:rcoigt:'otor,rtollitgarrorllattoig.'t.o:::''or:iit,oo:lal'tio.::;'t:ogiot,,r,alaillalo:oo.ollo.!'cig,oll;orati:ot,'liort:'r.oott:,'ari'licat,'llilcit,rlict.oot,'t?ot,tat':orag,::lataorlal.oti't,'o::;,oio:ot,'ararrorloiar:tatillitilora;ati,la::r,illoror!o,,it.oitt:'tatato:,caorott;:'llat:o:ot'!catilc,'t::aicialir,:traorlat,:?,:illla,ooorilat':l;'iototlticiglr!oiillr:'t,'licg.o?:'it.o':'r!it'olll?'laor?r'tat,ot?t,'col,:oc!tilrallortior?'tatoairarit.oorio,'lalg:alallliot,:ircit?:oit:ti,orrrcorr:?'t,iolatllag:cic.otg,?iloorortocit,o,:agratcit.lit:,ralilit,'ill;:r:oitataalti:ot;ot:ocit;rioorgir:t,atr:.;itictio:,or,orillitato'it,:aligatgoottarrto'':'olt::ociorllo?or,ot!,'tr'iotito:'llort?ig,'l?:'it,oolicg.:ot?.ratoticl:.iat!atrt;?oli't:,'tig,:,:ollot,',oi

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [47]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:t,liliirat,'cacloloricilcl''llo::.:tor,o!'lolitoitit.ig:attit?'l':'iit,oto?raca'llalllaig.'llt.'cat:
